In [ ]:
# ----- NOTES -----
# - pg.85: most common supervised learning tasks are regression and classification
# - in this chapter we are using the MNIST dataset, what is this?
# - pg.86: what is the typical dictionary structure for sklearn datasets
# - pg.86: what does matplotlib's imshow() method do?
# - pg.86: what is a feature vector?

In [7]:
# ----- IMPORTS -----
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
# here we are fetching the MNIST dataset
# DECSR key - describing the dataset
# data key - contains an array with one row per instance and one column per feature
# target key - containing an array with labels
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [3]:
# looking at these arrays
X, y = mnist['data'], mnist['target']

In [4]:
# this means there are 7,000 images and 784 features
# it's 784 because its 28 by 28 pixels, and each feature represents
# one pixels intensity
X.shape

(70000, 784)

In [5]:
#label
y.shape

(70000,)

In [6]:
28 * 28

784

In [ ]:
# here we are grabbing a feature vector and reshaping into a 28x28 array
# we then display it using matplotlib's imshow() function
